# **Exploração Visual das Correspondências de Cândido Portinari através de Metadados Textuais**
#### **por Micaele Brandão e Mateus Bandeira**

### **Tópicos**
1. Introdução
2. Configuração de ambiente
3. Coleta e limpeza de dados
4. Análise de redes
5. Panorama estatístico e temporal
6. Conclusão e Referências


### 1. Introdução

### 2. Configuração de ambiente

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

### 3. Coleta e limpeza de dados

Este código começa carregando dois arquivos distintos: o dados_docvirt.csv, que está mais completo, e o dados_extraidos.csv, que contém informações novas, mas em um formato bagunçado. Primeiro o script seleciona apenas as colunas essenciais do arquivo de extraídos (título e conteúdo), descarta o que é inútil e cria manualmente as colunas que faltam (como 'página' e 'status') para que ele espelhe exatamente a estrutura do arquivo principal. Com os dois conjuntos de dados agora organizados e com as colunas na mesma ordem, o código realiza a concatenação, empilhando as informações do DocVirt sobre as novas extrações. Para garantir que a lista final seja limpa, ele executa uma filtragem de duplicatas baseada nos títulos: se um mesmo item aparecer nos dois arquivos, o script mantém apenas a versão do DocVirt. Ao final, o programa exibe um resumo estatístico da operação para que você saiba quantos registros foram processados e salva o resultado consolidado em um novo arquivo chamado dados_consolidados_completo.csv.

In [2]:
import pandas as pd

# carregar os arquivos
# 'on_bad_lines' pra pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_docvirt.csv')
df_extraidos = pd.read_csv('dados_extraidos.csv')

# ajustar 'dados_extraidos' 
# o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# adc as colunas que faltam para ficar no padrão do docvirt
# se não houver informação, deixamos como None ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' 

# reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# juntar e completar
# concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo 
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# remover duplicatas
# se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# visualziar e salvar
print(f"Total de linhas no 'DocVirt' original: {len(df_docvirt)}")
print(f"Total de linhas no 'Extraídos' original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# primeiras linhas
display(df_final.head())

# salva como um csv
df_final.to_csv('dados_consolidados_completo.csv', index=False)
print("Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!")
df_extraidos['status'] = 'extraido_novo' 

Total de linhas no 'DocVirt' original: 9126
Total de linhas no 'Extraídos' original: 9126
Total de linhas após juntar e limpar: 5803


,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\r\nNUM_ANTDOC: 1\r\nPAGINAÇÃO: \r...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\r\nNUM_ANTDOC: 2\r\nPAGINAÇÃO: \r...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\r\nNUM_ANTDOC: 3\r\nPAGINAÇÃO: \r...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\r\nNUM_ANTDOC: 4\r\nPAGINAÇÃO: \r...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\r\nNUM_ANTDOC: 5\r\nPAGINAÇÃO: \r...,sucesso


Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!
